<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Aprendizado de Máquina, Classificação
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Classificação;</li>
  <li>Dados;</li>
  <li>Treino;</li>
  <li>Avaliação;</li>
  <li>Predição.</li>
</ol>

---

# **Exercícios**

## 1\. Pinguins

Neste exercício, vamos utilizar uma base de dados com informações sobre penguins. A idéia é prever a espécie do penguin (**species**) baseado em suas características físicas e geográficas (variáveis preditivas).

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
penguim = sns.load_dataset('penguins')

In [ ]:
penguim.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female


### **1.1. Análise exploratória**

Utilize os gráficos abaixo para entender melhor a relação entre os atributos e variável resposta da base de dados. Comente o que observou em cada gráfico.

 - Atributos numéricos por espécie:

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.pairplot(data=penguim.drop(['sex', 'island'], axis=1), hue="species", palette="pastel")

**Comentário:** ?

 - Sexo por espécie:

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.countplot(data=penguim, x='sex', hue="species", palette="pastel")

**Comentário:** ?

 - Ilha por espécie:

In [ ]:
with sns.axes_style('whitegrid'):

  grafico = sns.countplot(data=penguim, x='island', hue="species", palette="pastel")

**Comentário:** ?

## 2\. Dados

### **2.1. Valores nulos**

A base de dados possui valores faltantes, utilize os conceitos da aula para trata-los.

In [ ]:
# resposta da questão 2.1
# verificar se á dados faltantes
dados_faltantes_total = penguim.isnull().any()
print( dados_faltantes_total)

In [ ]:
# Para lidar com dados faltantes uma das opções e descarta as linha onde
# há dados faltantes.
penguim_df_limp = penguim.dropna()

In [ ]:
dt_teste = penguim_df_limp.isnull().any()
print(dt_teste)

### **2.2. Variáveis categóricas**

Identifique as variáveis categóricas nominais e ordinais, crie uma nova coluna aplicando a técnica correta de conversão a seus valores. A nova coluna deve ter o mesmo nome da coluna original acrescidade de "*_nom*" ou "*_ord*".

> **Nota**: Você não deve tratar a variável resposta.

> **Nota**: Por definição, árvores de decisão **não precisam** da transformação de atributos categóricos em numéricos. Contudo, por **limitação** do pacote Python Scikit Learn, devemos conduzir esta etapa. Mais informações neste [link](https://scikit-learn.org/stable/modules/tree.html#tree).

In [ ]:
# resposta da questão 2.2

# respota mais aceitavel para a coluna "island"

islanda_nom = pd.get_dummies(penguim_df_limp['island'])
islanda_nom.columns = [ str(col) + '_nrm'  for col in islanda_nom.columns]

penguim_df_limp = pd.concat([penguim_df_limp,islanda_nom], axis=1)

# Repetir o processo com a coluna "sex"

sex_m = pd.get_dummies(penguim_df_limp['sex'])
sex_m.columns = [ str(col) + '_nrm'  for col in sex_m.columns]

penguim_df_limp = pd.concat([penguim_df_limp,sex_m], axis=1)

# Repetir o processo com a coluna "species"

especie_nom = pd.get_dummies(penguim_df_limp['species'])
especie_nom.columns = [str(col) + '_nrm' for col in especie_nom.columns]

penguim_df_limp = pd.concat([penguim_df_limp,especie_nom], axis=1)

### **2.3. Limpeza**

Descarte as colunas categóricas originais e mantenha a variável resposta na primeira coluna do dataframe.

In [ ]:
# resposta da questão 2.3
penguim_df_limp = penguim_df_limp.drop(columns=['island','bill_length_mm','bill_depth_mm','flipper_length_mm','sex'],axis=1)

### **2.4. Treino/Teste**

Separe a base de dados em treino e teste utilizando uma proporção de 2/3 para treino e 1/3 para testes.

In [ ]:
# resposta da questão 2.4
predictors_train, predictors_test, target_train, target_test = train_test_split(
    penguim_df_limp.drop(['species'], axis=1),
    penguim_df_limp['species'],
    test_size=0.25,
    random_state=123
)

## 3\. Modelagem

### **3.1. Treino**

Treine um modelo de **árvore de decisão** com os **dados de treino** (2/3). Gere o gráfico da árvore do modelo treinado e responda: quantas **folhas** a árvore treinada possui?

**Resposta:** ?

In [ ]:
# resposta da questão 3.1
penguim_df_limp.head()

,body_mass_g,Biscoe_nrm,Dream_nrm,Torgersen_nrm,Female_nrm,Male_nrm,Adelie_nrm,Chinstrap_nrm,Gentoo_nrm
0,3750.0,0,0,1,0,1,1,0,0
1,3800.0,0,0,1,1,0,1,0,0
2,3250.0,0,0,1,1,0,1,0,0
4,3450.0,0,0,1,1,0,1,0,0
5,3650.0,0,0,1,0,1,1,0,0


### **3.2. Avaliação**

**a.** Matriz de Confusão

Calcule e visualize a **matriz de confusão** para o modelo de **árvore de decisão** treinado com os **dados de teste** (1/3). Comente os resultados.

**Comentário:** ?

In [ ]:
# resposta da questão 3.2.a

**b.** Acurácia

Calcule a **acurácia** para o modelo de **árvore de decisão** treinado com os **dados de teste** (1/3).

> **Nota:** Como referência, eu consegui uma acurácia de approx. 96% (sua acurácia pode não ser igual).

In [ ]:
# resposta da questão 3.2.b

## 4\. Predição  

### **4.1. Novo penguim**

Qual a espécie de um penguim com as seguintes características:


| island | bill_length_mm | bill_depth_mm | flipper_length_mm | body_mass_g | sex |
| --- | --- | --- | --- | --- | --- |
| Biscoe | 38.2 | 18.1 | 185.0 | 3950.0 | Male |

> **Atenção:** Lembre-se de pre-processar os atributos assim como nos exercício 2.2. A ordem dos atributos importa, deve ser a mesma usada na modelagem.

> **Nota:** Como referência eu obtive **adelie** como espécie predita (a sua predição pode não ser igual).

In [ ]:
# resposta da questão 4.1

---